In [4]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform,data
%matplotlib inline
import sys,os,caffe
import cv2
import copy
import math
import time
from scipy.ndimage import gaussian_filter
from numpy.lib.stride_tricks import as_strided as ast

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# RGB -> YCbCr
def rgb2ycbcr(rgb):
    m = np.array([[ 65.481, 128.553, 24.966],
                  [-37.797, -74.203, 112],
                  [ 112, -93.786, -18.214]])
    shape = rgb.shape
    if len(shape) == 3:
        rgb = rgb.reshape((shape[0] * shape[1], 3))
    ycbcr = np.dot(rgb, m.transpose() / 255.)
    ycbcr[:,0] += 16.
    ycbcr[:,1:] += 128.
    return ycbcr.reshape(shape)

def ycbcr2rgb(ycbcr):
    m = np.array([[ 65.481, 128.553, 24.966],
                  [-37.797, -74.203, 112],
                  [ 112, -93.786, -18.214]])
    shape = ycbcr.shape
    if len(shape) == 3:
        ycbcr = ycbcr.reshape((shape[0] * shape[1], 3))
    rgb = copy.deepcopy(ycbcr)
    rgb[:,0] -= 16.
    rgb[:,1:] -= 128.
    rgb = np.dot(rgb, np.linalg.inv(m.transpose()) * 255.)
    return rgb.clip(0, 255).reshape(shape)

def psnr(img1, img2, boundary):
    HR = img1[:,boundary:img1.shape[0]-boundary,boundary:img1.shape[1]-boundary]
    LR = img2[:,boundary:img2.shape[0]-boundary,boundary:img2.shape[1]-boundary]
    mse = np.mean( (HR - LR) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def merge_img_tensor(list_img, height, width, patch_size, stride, upscale):

    patch_size = upscale * patch_size
    stride = upscale * stride
    height = upscale * height
    width = upscale * width
    out = np.zeros((height, width, 5)).astype(np.float32)
    mask = np.zeros((height, width, 5)).astype(np.float32)

    for i in range(0, height, stride):
        for j in range(0, width, stride):
            x = min(height, i + patch_size)
            y = min(width, j + patch_size)
            mask_patch = np.zeros((5,height, width)).astype(np.float32)
            out_patch = np.zeros((5,height, width)).astype(np.float32)
            mask_patch[(x - patch_size):x, (y - patch_size):y, :] = 1.0
            out_patch[(x - patch_size):x, (y - patch_size):y, :] = list_img.pop(0)
            mask = mask + mask_patch
            out = out + out_patch

    out = out / mask
    return out


def split_img_tensor(img, height, width, patch_size, stride, upscale):
    # resize image and convert them into tensor

    patch_box = np.empty((0))

    # split image into over-lapping pieces
    for i in range(0, height, stride):
        for j in range(0, width, stride):
            x = min(height, i + patch_size)
            y = min(width, j + patch_size)
            part = img[:, x-patch_size:x, y-patch_size:y]

            patch_box = np.append(patch_box,part)

    return patch_box

caffe_root = '/home/server/caffe/' 
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)

    
caffe.set_mode_gpu()


model_def = caffe_root + 'examples/DecisionTree_CNN/extra/image/video/ST-CNN_Deploy.prototxt'
image_root = caffe_root + 'examples/DecisionTree_CNN/extra/image/video/Vid4/walk/'

video_1 = np.zeros((576,720,3)).astype(np.double)
video_2 = np.zeros((576,704,3)).astype(np.double)
video_3 = np.zeros((480,720,3)).astype(np.double)
video_4 = np.zeros((480,720,3)).astype(np.double)



    

Model_root = 'examples/DecisionTree_CNN/extra/image/video/Train_iter_X4.caffemodel'

model_weights = caffe_root + Model_root

net = caffe.Net(model_def, model_weights, caffe.TEST)
channel_swap = (1, 2, 0)
shape =[480,720]
LR_frame = np.zeros((5,120,180)).astype(np.float32)
HR_frame = np.zeros((shape[0],shape[1],5)).astype(np.float32)
SR_frame = np.zeros((5,shape[0],shape[1],3)).astype(np.float32)
psnr_ave = 0
upscale = 4
patch_size = 120
stride = 120
start = time.time()

for i in range(0,45,5):
    for j in range(0,5,1):
        HR = cv2.imread(image_root + "HR/" + str(i+j+1).zfill(4) + ".png")
        HR_float = np.float32(HR[...,::-1])
        HR_yuv = rgb2ycbcr(HR_float) / 255.0
        HR_frame[:,:,j] = HR_yuv[:,:,0]
        LR = cv2.imread(image_root + "LR/" + str(i+j+1).zfill(4) + ".png")
        LR_float = np.float32(LR[...,::-1])
        LR_yuv = rgb2ycbcr(LR_float) / 255.0
        LR_frame[j,:,:] = LR_yuv[:,:,0]
        uv_frame = cv2.resize(LR_yuv,(0,0),fx=4,fy=4, interpolation = cv2.INTER_CUBIC)
        SR_frame[j,:,:,1] = uv_yuv[:,:,1]
        SR_frame[j,:,:,2] = uv_yuv[:,:,2]
    
        
    SR_output = np.zeros((shape[0], shape[1], 5)).astype(np.float32)
    mask = np.zeros((shape[0], shape[1], 5)).astype(np.float32)

    for m in range(0, shape[0], upscale*stride):
        for n in range(0, shape[1], upscale*stride):
            x = min(shape[0], m + patch_size*upscale)
            y = min(shape[1], n + patch_size*upscale)
            patch = LR_frame[:, x/upscale-patch_size:x/upscale, y/upscale-patch_size:y/upscale]
            net.blobs['data'].data[...] = patch
            net.forward()
            patch_SR = net.blobs['SR_final'].data[0,:,:,:].transpose(channel_swap)
            mask_patch = np.zeros((shape[0], shape[1], 5)).astype(np.float32)
            out_patch = np.zeros((shape[0], shape[1], 5)).astype(np.float32)
            mask_patch[(x - patch_size*upscale):x, (y - patch_size*upscale):y, :] = 1.0
            out_patch[(x - patch_size*upscale):x, (y - patch_size*upscale):y, :] = patch_SR
            mask = mask + mask_patch
            SR_output = SR_output + out_patch

    SR_output = SR_output / mask
    value = psnr(HR_frame*255.0, SR_output*255.0, 4)
    psnr_ave = psnr_ave + value/8
    
    for k in range(0,5,1):
        SR = SR_frame[k,:,:,:]
        SR[:,:,0] = SR_output[:,:,k]
        SR_RGB = ycbcr2rgb(SR*255)
        #SR_Y = SR_output[:,:,k]*255.0
        cv2.imwrite(image_root + "SR/" + str(i+k+1).zfill(4) + ".png",SR_RGB[...,::-1])

end = time.time()

cost = end - start
print "time cost: ", cost
print "average PSNR is ", psnr_ave




time cost:  6.06119203568
average PSNR is  32.9672556821
